# abc_load.py Module Demo Notebook

This notebook demonstrates how to use the functions inside the abc_load.py module.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import abc_load as abc

## Load TH+ZI spatial subset from public ABC dataset

In [3]:
adata_th_zi = abc.load_adata(transform='log2', subset_to_TH_ZI=True, 
                             with_metadata=True, flip_y=True,
                             with_colors=True)

In [4]:
adata_th_zi

## def plot_dataset()

In [5]:
import anndata as ad
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.lines import Line2D
%matplotlib inline

def plot_dataset(adata, column='class'):
    
    cell_df = adata.obs

    # define legend elements for the column colors
    color_col = column+'_color'
    categories = cell_df[column].unique()
    cat_color_mapping = dict(zip(categories, cell_df[color_col].unique()))
    # sort the dict & the categories list by category
    cat_color_mapping = dict(sorted(cat_color_mapping.items()))
    categories = sorted(categories)
    legend_elements = [Line2D([0], [0], 
                              lw=0, marker='o', markersize=10,
                              markerfacecolor=cat_color_mapping[cat],
                              color=cat_color_mapping[cat], 
                              label=cat) 
                       for cat in cat_color_mapping]
    
    z_sections = sorted(cell_df['z_reconstructed'].unique())
    
    n_col = 2
    n_row = int(np.ceil(len(z_sections) / n_col))
    fig, axes = plt.subplots(n_row, n_col, figsize=(12,30))
    axes = axes.ravel()

    # x_min = 2.5; x_max = 8.5; y_min = 7.5; y_max = 4

    for i, sec in enumerate(z_sections):
        ax = axes[i]

        curr_sec_cell_df = cell_df[(cell_df['z_reconstructed']==sec)]

        sc = ax.scatter(curr_sec_cell_df['x_reconstructed'], curr_sec_cell_df['y_reconstructed'],
                        color=curr_sec_cell_df['class_color'],
                        s=0.5, marker='.', zorder=1)
        ax.set_title('z='+str(sec))

        # ax.set_xlim((x_min,x_max))
        # ax.set_ylim((y_min,y_max))
        ax.set_xlabel('x_reconstructed')
        ax.set_ylabel('y_reconstructed')
        ax.axis('equal')
        ax.axis('off')
        # ax.set_xticks([])
        # ax.set_yticks([])

        if i==7:
            ax.legend(legend_elements, categories, title='class', 
                      loc='center left', bbox_to_anchor=(1.1,0.5))

## Display output of abc.load_adata()

### display all cells

In [6]:
plot_dataset(adata_th_zi, column='class')

### display just neurons

In [7]:
nn_classes = ['30 Astro-Epen', '31 OPC-Oligo', '32 OEC', '33 Vascular', '34 Immune']
plot_dataset(adata_th_zi[~adata_th_zi.obs['class'].isin(nn_classes)], column='class')

## Cleanup loaded dataset with abc.filter_adata_by_class()

### keep only TH, ZI, & MH+LH classes

In [8]:
adata_th_zi_filter_all = abc.filter_adata_by_class(adata_th_zi, 
                                                  filter_nonneuronal=True, 
                                                  filter_midbrain=True)

plot_dataset(adata_th_zi_filter_all, column='class')

### keep midbrain classes as well

In [9]:
adata_th_zi_keep_MB = abc.filter_adata_by_class(adata_th_zi, 
                                                filter_nonneuronal=True, 
                                                filter_midbrain=False)

plot_dataset(adata_th_zi_keep_MB, column='class')

## Convert cell type labels between versions

### convert clusters

clusters are unique in the cluster_to_cluster_membership_pivoted.csv pivot table and are easily handled by abc.convert_taxonomy_labels()

In [10]:
# user input parameters
input_version = '20230630'
output_version = '20230830'

cl_labels_list = ['1130 TH Prkcd Grin2c Glut_1',
                   '1133 TH Prkcd Grin2c Glut_10', 
                   '1136 TH Prkcd Grin2c Glut_12', '1137 TH Prkcd Grin2c Glut_12',
                   '1138 TH Prkcd Grin2c Glut_13', '1142 TH Prkcd Grin2c Glut_13',
                   '1157 TH Prkcd Grin2c Glut_6', '1158 TH Prkcd Grin2c Glut_6',
                   '1159 TH Prkcd Grin2c Glut_6', 
                   '1168 TH Prkcd Grin2c Glut_8'
              ]
taxonomy_level = 'cluster'

In [11]:
converted_cl_list = abc.convert_taxonomy_labels(cl_labels_list, taxonomy_level, 
                                                label_format='id_label',
                                                input_version='20230630', 
                                                output_version='20230830',
                                                output_as_dict=False)
converted_cl_list

In [12]:
converted_cl_dict = abc.convert_taxonomy_labels(cl_labels_list, taxonomy_level, 
                                                label_format='id_label',
                                                input_version='20230630', 
                                                output_version='20230830',
                                                output_as_dict=True)
converted_cl_dict

### convert supertypes

supertypes, subclasses, and classes are not unique in the cluster_to_cluster_membership_pivoted.csv pivot table and are handled slightly differently by abc.convert_taxonomy_labels() to account for this

In [13]:
st_labels_list = [
                   '0270 TH Prkcd Grin2c Glut_1',
                   '0271 TH Prkcd Grin2c Glut_10',
                   # '0280 TH Prkcd Grin2c Glut_6'
                  ]
taxonomy_level = 'supertype'

In [14]:
converted_st_list = abc.convert_taxonomy_labels(st_labels_list, taxonomy_level, 
                                                label_format='id_label',
                                                input_version='20230630', 
                                                output_version='20230830',
                                                output_as_dict=False)
converted_st_list

In [15]:
converted_st_dict = abc.convert_taxonomy_labels(st_labels_list, taxonomy_level, 
                                                label_format='id_label',
                                                input_version='20230630', 
                                                output_version='20230830',
                                                output_as_dict=True)
converted_st_dict

## Get color dictionary for a list of cell types

### v20230630 colors

In [16]:
taxonomy_level = 'cluster'
cl_color_dict = abc.get_color_dictionary(cl_labels_list, 
                                                   taxonomy_level, 
                                                   label_format='id_label',
                                                   version='20230630')
cl_color_dict

In [17]:
taxonomy_level = 'supertype'
st_color_dict = abc.get_color_dictionary(st_labels_list, 
                                           taxonomy_level, 
                                           label_format='id_label',
                                           version='20230630')
st_color_dict

### v20230830 colors

In [18]:
taxonomy_level = 'cluster'
converted_cl_color_dict = abc.get_color_dictionary(converted_cl_list, 
                                                   taxonomy_level, 
                                                   label_format='id_label',
                                                   version='20230830')
converted_cl_color_dict

In [19]:
taxonomy_level = 'supertype'
converted_st_color_dict = abc.get_color_dictionary(converted_st_list, 
                                                       taxonomy_level, 
                                                       label_format='id_label',
                                                       version='20230830')
converted_st_color_dict